In [1]:
# Filename: train_mnist_cnn.py (or in a Colab notebook)

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

# ===========================
# 1. Configuration
# ===========================
BATCH_SIZE = 64
EPOCHS = 5
LR = 0.001
MODEL_PATH = "mnist_cnn.pth"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ===========================
# 2. Dataset
# ===========================
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


In [2]:

train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)


100%|██████████| 9.91M/9.91M [00:00<00:00, 18.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 481kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.56MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.78MB/s]


In [3]:

# ===========================
# 3. Model Definition
# ===========================
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1, 1),  # [B, 1, 28, 28] → [B, 32, 28, 28]
            nn.ReLU(),
            nn.MaxPool2d(2),           # [B, 32, 14, 14]
            nn.Conv2d(32, 64, 3, 1, 1), # [B, 64, 14, 14]
            nn.ReLU(),
            nn.MaxPool2d(2),           # [B, 64, 7, 7]
            nn.Flatten(),              # [B, 64*7*7]
            nn.Linear(64*7*7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)         # Output layer for 10 digits
        )

    def forward(self, x):
        return self.model(x)

model = SimpleCNN().to(DEVICE)



In [4]:
# ===========================
# 4. Loss and Optimizer
# ===========================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

# ===========================
# 5. Training Loop
# ===========================
print("Training Started...")
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {running_loss/len(train_loader):.4f}")

# ===========================
# 6. Save the Model
# ===========================
os.makedirs("models", exist_ok=True)
torch.save(model.state_dict(), f"models/{MODEL_PATH}")
print(f"Model saved to models/{MODEL_PATH}")


Training Started...
Epoch [1/5], Loss: 0.1537
Epoch [2/5], Loss: 0.0461
Epoch [3/5], Loss: 0.0317
Epoch [4/5], Loss: 0.0225
Epoch [5/5], Loss: 0.0158
Model saved to models/mnist_cnn.pth
